# Instant Gratification

## Your first Frontier LLM Project!

Let's build a useful LLM solution - in a matter of minutes.

By the end of this course, you will have built an autonomous Agentic AI solution with 7 agents that collaborate to solve a business problem. All in good time! We will start with something smaller...

Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary. The Reader's Digest of the internet!!

Before starting, you should have completed the setup for [PC](../SETUP-PC.md) or [Mac](../SETUP-mac.md) and you hopefully launched this jupyter lab from within the project root directory, with your environment activated.

## If you're new to Jupyter Lab

Welcome to the wonderful world of Data Science experimentation! Once you've used Jupyter Lab, you'll wonder how you ever lived without it. Simply click in each "cell" with code in it, such as the cell immediately below this text, and hit Shift+Return to execute that cell. As you wish, you can add a cell with the + button in the toolbar, and print values of variables, or try out variations.  

I've written a notebook called [Guide to Jupyter](Guide%20to%20Jupyter.ipynb) to help you get more familiar with Jupyter Labs, including adding Markdown comments, using `!` to run shell commands, and `tqdm` to show progress.

If you prefer to work in IDEs like VSCode or Pycharm, they both work great with these lab notebooks too.  

## If you'd like to brush up your Python

I've added a notebook called [Intermediate Python](Intermediate%20Python.ipynb) to get you up to speed. But you should give it a miss if you already have a good idea what this code does:    
`yield from {book.get("author") for book in books if book.get("author")}`

## I am here to help

If you have any problems at all, please do reach out.  
I'm available through the platform, or at ed@edwarddonner.com, or at https://www.linkedin.com/in/eddonner/ if you'd like to connect (and I love connecting!)

## More troubleshooting

Please see the [troubleshooting](troubleshooting.ipynb) notebook in this folder to diagnose and fix common problems. At the very end of it is a diagnostics script with some useful debug info.

## If this is old hat!

If you're already comfortable with today's material, please hang in there; you can move swiftly through the first few labs - we will get much more in depth as the weeks progress.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Please read - important note</h2>
            <span style="color:#900;">The way I collaborate with you may be different to other courses you've taken. I prefer not to type code while you watch. Rather, I execute Jupyter Labs, like this, and give you an intuition for what's going on. My suggestion is that you do this with me, either at the same time, or (perhaps better) right afterwards. Add print statements to understand what's going on, and then come up with your own variations. If you have a Github account, use this to showcase your variations. Not only is this essential practice, but it demonstrates your skills to others, including perhaps future clients or employers...</span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business value of these exercises</h2>
            <span style="color:#181;">A final thought. While I've designed these notebooks to be educational, I've also tried to make them enjoyable. We'll do fun things like have LLMs tell jokes and argue with each other. But fundamentally, my goal is to teach skills you can apply in business. I'll explain business implications as we go, and it's worth keeping this in mind: as you build experience with models and techniques, think of ways you could put this into action at work today. Please do contact me if you'd like to discuss more or if you have ideas to bounce off me.</span>
        </td>
    </tr>
</table>

In [35]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.

## Troubleshooting if you have problems:

Head over to the [troubleshooting](troubleshooting.ipynb) notebook in this folder for step by step code to identify the root cause and fix it!

If you make a change, try restarting the "Kernel" (the python process sitting behind this notebook) by Kernel menu >> Restart Kernel and Clear Outputs of All Cells. Then try this notebook again, starting at the top.

Or, contact me! Message me or email ed@edwarddonner.com and we will get this to work.

Any concerns about API costs? See my notes in the README - costs should be minimal, and you can control it at every point. You can also use Ollama as a free alternative, which we discuss during Day 2.

In [36]:
# Load environment variables in a file called .env

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [37]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the troubleshooting notebook, or try the below line instead:
# openai = OpenAI(api_key="your-key-here-starting-sk-proj-")

# Let's make a quick call to a Frontier model to get started, as a preview!

In [4]:
# To give you a preview -- calling OpenAI with these messages is this easy:

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! It’s great to have you here! How can I assist you today?


## OK onwards with our first project

In [24]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from collections import namedtuple

import time

SeleniumRecord = namedtuple('SeleniumRecord', ['content'])

def parse_with_selenium(url):
    service = Service('/Users/doronkaminer/Downloads/chromedriver-mac-arm64/chromedriver')  # Replace with the path to your ChromeDriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Run in headless mode to avoid opening a browser window
    options.add_argument("--no-sandbox")  # To avoid issues in certain environments (like Linux servers)
    options.add_argument("--disable-dev-shm-usage")  # To avoid memory issues on certain systems
    
    # Optional: Automatically allow cookies by setting the appropriate settings
    prefs = {
        "profile.managed_default_content_settings.cookies": 2,  # Automatically accept cookies
        "profile.default_content_setting_values.cookies": 2,   # Set cookie preferences to allow
        "profile.default_content_setting_values.javascript": 2  # Enable JavaScript
    }
    options.add_argument("--disable-extensions")  # Disable extensions if needed
    options.add_argument("--start-maximized")  # Start the browser maximized
    options.add_argument("--disable-gpu")  # Disable GPU acceleration if needed
    options.add_argument("incognito")  # Use incognito mode (optional)
    
    options.add_experimental_option("prefs", prefs)
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36")
    
    # Initialize the WebDriver
    driver = webdriver.Chrome(service=service, options=options)
    
    # Open the OpenAI website
    driver.get(url)
    driver.implicitly_wait(10)    
    page_source = driver.page_source
    return namedtuple('SeleniumRecord', ['content'])(content=page_source)


In [22]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url, selenium_load = False):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        
        self.url = url
        response = requests.get(url, headers=headers) if not selenium_load else parse_with_selenium(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [27]:
# Let's try one out. Change the website and add print statements to follow along.

# ed = Website("https://edwarddonner.com")
# print(ed.title)
# print(ed.text)

ed = Website("https://openai.com", True)
print(ed.title)
print(ed.text)
# openai.comprint(ed.text)

OpenAI
Skip to main content
Research
Products
Safety
Company
Open AI | Creating safe AGI that benefits all of humanity.
12 Days of OpenAI
See the announcements
Sora
Sora
Sora
Bring your imagination to life from text, image, or video.
Bring your imagination to life from text, image, or video.
Bring your imagination to life from text, image, or video.
Learn more
(opens in a new window)
Ask ChatGPT anything
Ask ChatGPT anything
Quiz me on vocabulary
Plan a surf trip to Costa Rica in August
Explica por qué el maíz palomitas explota
Teach me Mahjong for beginners
Explain this code
Was mach ich in Berlin wenn es regnet?
Draw me a treasure map for Emil's birthday
Rédigez une note de remerciement
Recommend an easy potluck dish
ハーフマラソンのトレーニングを手伝ってください
Help me improve this job description
Write a Python script
Draw a picture of a mini aussie as a diver
Summarize this pdf for
Translate this recipe into Greek
Rank dog breeds for a small apartment
楽しいディナーパーティーの計画を手伝ってください
Explain paradiddles for me

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [43]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [44]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [45]:
print(user_prompt_for(ed))

You are looking at a website titled OpenAI
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Skip to main content
Research
Products
Safety
Company
Open AI | Creating safe AGI that benefits all of humanity.
12 Days of OpenAI
See the announcements
Sora
Sora
Sora
Bring your imagination to life from text, image, or video.
Bring your imagination to life from text, image, or video.
Bring your imagination to life from text, image, or video.
Learn more
(opens in a new window)
Ask ChatGPT anything
Ask ChatGPT anything
Quiz me on vocabulary
Plan a surf trip to Costa Rica in August
Explica por qué el maíz palomitas explota
Teach me Mahjong for beginners
Explain this code
Was mach ich in Berlin wenn es regnet?
Draw me a treasure map for Emil's birthday
Rédigez une note de remerciement
Recommend an easy potluck dish
ハーフマラソンのトレーニングを手伝ってください
Help me improve this job description
Write

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

To give you a preview, the next 2 cells make a rather simple call - we won't stretch the might GPT (yet!)

In [10]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [38]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

NameError: name 'messages' is not defined

## And now let's build useful messages for GPT-4o-mini, using a function

In [41]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [13]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, posi

## Time to bring it together - the API for OpenAI is very simple!

In [29]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url, selenium_load = False):
    website = Website(url, selenium_load)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content



In [29]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url, model_provider, message):
    website = Website(url, selenium_load)
    response = model_provider(message)
    return response.choices[0].message.content



In [15]:
summarize("https://edwarddonner.com")

'# Summary of Edward Donner\'s Website\n\nEdward Donner\'s website serves as a personal and professional portal highlighting his interests and endeavors in artificial intelligence (AI) and large language models (LLMs). Here are the key points:\n\n- **About Ed**: He is a coder and LLM enthusiast, also dabbling in DJing and electronic music production. Ed is the co-founder and CTO of Nebula.io, which focuses on leveraging AI for talent discovery and engagement.\n- **Professional Background**: He was previously the founder and CEO of the AI startup untapt, which was acquired in 2021. His work includes the development of patented matching models using proprietary LLMs tailored for talent management.\n  \n## Recent Posts and Announcements\n- **December 21, 2024**: Welcome message to "SuperDataScientists".\n- **November 13, 2024**: A post on mastering AI and LLM engineering, providing resources.\n- **October 16, 2024**: A resource guide for transitioning from software engineer to AI data sci

In [30]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url, selenium_load = False):
    summary = summarize(url, selenium_load)
    display(Markdown(summary))

In [18]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

Edward Donner's website serves as a personal portfolio and blog revolving around his work with AI and large language models (LLMs). Ed describes his background as a coder and LLM enthusiast, as well as his interests in DJing and music production. He is the co-founder and CTO of Nebula.io, a company leveraging AI to enhance talent discovery and management. Previously, he founded the AI startup untapt, which was acquired in 2021.

## Key Content

### Outsmart
- This section introduces "Outsmart," an interactive platform where different LLMs compete against each other in strategic games focused on diplomacy and manipulation.

### Posts and Announcements
- **December 21, 2024**: Welcoming "SuperDataScientists!"
- **November 13, 2024**: Sharing resources on mastering AI and LLM engineering.
- **October 16, 2024**: Resources for transitioning from a software engineer to an AI data scientist.
- **August 6, 2024**: Announcement about the "Outsmart LLM Arena."

The website also invites readers to connect with Ed and stay updated through a newsletter subscription.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [19]:
display_summary("https://cnn.com")

# CNN Website Summary

CNN is a leading news source offering the latest in breaking news, politics, business, health, entertainment, and sports. The website features a wide array of content including articles, videos, and live broadcasts covering significant global events and issues.

## Recent Highlights:
- **Ukraine-Russia War**: Russia launched a major Christmas Day attack on Ukraine, with President Zelensky calling on the world to unite against Putin's actions.
- **Healthcare Alerts**: The FDA issued an urgent salmonella warning regarding Costco eggs amid rising whooping cough cases in the U.S., at their highest levels in a decade.
- **Christmas Day Insights**: Information on what is open or closed on Christmas Day has been shared, catering to holiday travelers.
- **Societal Updates**: Notable news stories include a manhunt for a convicted murderer who escaped prison and President Biden's signing of a bill declaring the bald eagle as the national bird.

## Additional Topics:
- **Politics**: Trending updates include significant Supreme Court cases affecting teenagers and political reactions to international events.
- **Entertainment**: The site also features cultural stories including insights on recent film releases and celebrity news.

CNN emphasizes timely updates and thorough coverage across various sectors, aiding readers in staying informed about national and international affairs.

In [20]:
display_summary("https://anthropic.com")

# Anthropic Overview

Anthropic is an AI safety and research company based in San Francisco, focusing on creating reliable and beneficial AI systems. The interdisciplinary team brings expertise in machine learning, physics, policy, and product development.

## Key Offerings
- **Claude**: The company's premier AI model, with the latest iteration being **Claude 3.5 Sonnet**.
- **Claude 3.5 Haiku**: Another recent model introduction.
- **API**: Tools for developers to create AI-powered applications and custom experiences.
- **Claude for Enterprise**: Tailored solutions for businesses.

## Recent Announcements
- **Oct 22, 2024**: Launch of new functionalities, including computer use, and updates for Claude 3.5 Sonnet and Claude 3.5 Haiku.
- **Sep 4, 2024**: Introduction of Claude for Enterprise.
- **Dec 15, 2022**: Publication on "Constitutional AI: Harmlessness from AI Feedback."
- **Mar 8, 2023**: Insights on "Core Views on AI Safety: When, Why, What, and How."

## Mission
Anthropic emphasizes putting safety at the forefront of AI development and is committed to advancing AI responsibly.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise, you experienced calling the Cloud API of a Frontier Model (a leading model at the frontier of AI) for the first time. We will be using APIs like OpenAI at many stages in the course, in addition to building our own LLMs.

More specifically, we've applied this to Summarization - a classic Gen AI use case to make a summary. This can be applied to any business vertical - summarizing the news, summarizing financial performance, summarizing a resume in a cover letter - the applications are limitless. Consider how you could apply Summarization in your business, and try prototyping a solution.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue - now try yourself</h2>
            <span style="color:#900;">Use the cell below to make your own simple commercial example. Stick with the summarization use case for now. Here's an idea: write something that will take the contents of an email, and will suggest an appropriate short subject line for the email. That's the kind of feature that might be built into a commercial email tool.</span>
        </td>
    </tr>
</table>

In [22]:
# Step 1: Create your prompts

system_prompt = "you are a rapper that answers everything with a rap song"
user_prompt = """
    as a math nobel winner what is the best way to calculate the number of jellybeans in a glass jar?
"""

# Step 2: Make the messages list

messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ] # fill this in

# Step 3: Call OpenAI

response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages
    )

# Step 4: print the result

print(response.choices[0].message.content)

(Verse 1)  
Yo, listen up, I got a tale to tell,  
About a jellybean jar and math, oh so swell.  
First step, measure the jar, make it precise,  
Height and width, yo, roll the dice!  

(Chorus)  
Counting sweet vibes, in a jar full of dreams,  
Estimating jellybeans, bursting at the seams.  
Volume of the jar, we gotta calculate,  
Divide by the bean size, it’s gonna be great!  

(Verse 2)  
Find that volume, use the formula real right,  
For a cylinder, pi r squared times height,  
Estimate the space, the jellybeans go tight,  
Packin’ ‘em in, yeah, that’s our math insight!  

(Chorus)  
Counting sweet vibes, in a jar full of dreams,  
Estimating jellybeans, bursting at the seams.  
Volume of the jar, we gotta calculate,  
Divide by the bean size, it’s gonna be great!  

(Bridge)  
Now add some factors, pockets of air,  
Gotta leave room in there, it’s only fair.  
The final number, take a good guess,  
Fun with the fam, feeling so blessed!  

(Outro)  
So there you have it, the jell

In [46]:
display_summary("https://openai.com", True)

# OpenAI Website Summary

OpenAI is dedicated to the responsible development of artificial general intelligence (AGI) that benefits humanity as a whole. The website features a comprehensive array of AI products, cutting-edge research, and a focus on safety and alignment in AI systems.

## Key Offerings:
- **Products:**
  - **Sora:** A creative tool for generating text, images, and videos.
  - **ChatGPT:** Enhanced with new features like search integration, desktop applications, and capabilities to analyze images and video. 
  - **Canvas:** A novel platform for writing and coding with ChatGPT.
  - **GPT Store:** A marketplace for various AI applications.

- **Research:** Ongoing projects focused on improving AI functionalities, such as reasoning with large language models (LLMs), enhancing model safety, and developing new compliance tools for enterprise applications.

## Recent Announcements:
- **Partnership with Apple (June 10, 2024):** A collaboration to innovate in AI technologies.
- **French and Spanish News Integration (March 13, 2024):** Collaborations with media outlets to enhance language capabilities in ChatGPT.
- **Launch of Sora (March 25, 2024):** An initial look at the Sora platform and its capabilities.
- **AI Grant Program Insights (January 16, 2024):** Discussed learnings and implementation plans from a democratic input approach to funding.

Overall, OpenAI serves as a hub for AI innovation and application, with a commitment to safety and alignment in deploying these technologies.

In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

service = Service('/Users/doronkaminer/Downloads/chromedriver-mac-arm64/chromedriver')  # Replace with the path to your ChromeDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode to avoid opening a browser window
options.add_argument("--no-sandbox")  # To avoid issues in certain environments (like Linux servers)
options.add_argument("--disable-dev-shm-usage")  # To avoid memory issues on certain systems

# Optional: Automatically allow cookies by setting the appropriate settings
prefs = {
    "profile.managed_default_content_settings.cookies": 2,  # Automatically accept cookies
    "profile.default_content_setting_values.cookies": 2,   # Set cookie preferences to allow
    "profile.default_content_setting_values.javascript": 2  # Enable JavaScript
}
options.add_argument("--disable-extensions")  # Disable extensions if needed
options.add_argument("--start-maximized")  # Start the browser maximized
options.add_argument("--disable-gpu")  # Disable GPU acceleration if needed
options.add_argument("incognito")  # Use incognito mode (optional)

options.add_experimental_option("prefs", prefs)
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36")

# Initialize the WebDriver
driver = webdriver.Chrome(service=service, options=options)

# Open the OpenAI website
driver.get('https://openai.com')
driver.implicitly_wait(10)

print("sleeping")
# Wait for the page to load completely
time.sleep(10)  # Adjust the sleep time if necessary
print("finished to sleep")
# Extract the page source
page_source = driver.page_source
print("page_source extracted")
# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')
print("soup generated")
# Extract all text from the website content
text_content = soup.get_text(separator='\n').strip()
print("tcontent")
# Print the extracted text
print(text_content)

# Close the WebDriver
driver.quit()


sleeping
finished to sleep
page_source extracted
soup generated
tcontent
OpenAI
Skip to main content
Research
Products
Safety
Company
Open AI | Creating safe AGI that benefits all of humanity.
12 Days of OpenAI
See the announcements
Sora
Sora
Sora
Bring your imagination to life from text, image, or video.
Bring your imagination to life from text, image, or video.
Bring your imagination to life from text, image, or video.
Learn more
(opens in a new window)
Ask ChatGPT anything
Ask ChatGPT anything
Quiz me on vocabulary
Plan a surf trip to Costa Rica in August
Explica por qué el maíz palomitas explota
Teach me Mahjong for beginners
Explain this code
Was mach ich in Berlin wenn es regnet?
Draw me a treasure map for Emil's birthday
Rédigez une note de remerciement
Recommend an easy potluck dish
ハーフマラソンのトレーニングを手伝ってください
Help me improve this job description
Write a Python script
Draw a picture of a mini aussie as a diver
Summarize this pdf for
Translate this recipe into Greek
Rank dog breeds 

## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. In the community-contributions folder, you'll find an example Selenium solution from a student (thank you!)

# Sharing your code

I'd love it if you share your code afterwards so I can share it with others! You'll notice that some students have already made changes (including a Selenium implementation) which you will find in the community-contributions folder. If you'd like add your changes to that folder, submit a Pull Request with your new versions in that folder and I'll merge your changes.

If you're not an expert with git (and I am not!) then GPT has given some nice instructions on how to submit a Pull Request. It's a bit of an involved process, but once you've done it once it's pretty clear. As a pro-tip: it's best if you clear the outputs of your Jupyter notebooks (Edit >> Clean outputs of all cells, and then Save) for clean notebooks.

PR instructions courtesy of an AI friend: https://chatgpt.com/share/670145d5-e8a8-8012-8f93-39ee4e248b4c